<a href="https://colab.research.google.com/github/A2Eweb/test/blob/master/RNAseq_tutorial_python_MASTER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this tutorial you will practice some first steps for exploring the results of an RNA sequencing experiment. In this notebook you will:

*   become familiar with the format of data produced by RNA sequencing
*   learn important normalization steps
*   conduct a principal component analysis

This notebook is not editable. To get started - make a copy of this notebook for yourself, and then edit away!


### The Experiment

We will be working with RNA sequencing data from an experiment measuring gene expression in the algae _Chromochloris zofingiensis_. 

<table><tr><td>

This algae is important because:  

+ it stores large amounts of energy from the sun,  which can then be turned into biofuel
+ it produces molecules that are beneficial for human health, like antioxidants</td>

<td>
<img src="https://phycocosm.jgi.doe.gov/public/Chrzof1/Roth_Algae2_TL+Pos+000+Time+115.jpg;jsessionid=208F5C1784C8309971029B88947B5595" width="250" ></td>

</tr>
</table>


Recently, scientists performed an experiment to figure out which genes were most important for these functions.  You can read more about the experiment [here](https://www.pnas.org/content/114/21/E4296). Specifically pay attention to the section titled "High Light-Induced Gene Expression", where scientists looked at which genes were 'turned on' (ie. increased their expression levels) when _C. zofingiensis_ samples were exposed to stronger light.

Why would it be helpful for scientists to know which genes were expressed when the algae was exposed to high light?

_add your answers here_

#### Download RNA-seq data

Now let's download the data.  The RNA-seq reads for each sample have already been aligned to a reference genome, and the number of reads matching each gene in each sample have been counted. Run the code below to download a text file containing a matrix of gene expression data for many different _C. zofingiensis_ samples. The matrix will be saved to a file in your google colab directory called 'rnaseq_raw_counts.txt'.

In [ ]:
file_id = "1ExdcAKMvn0QaEv4a8AaG1Dgy2TvlcVlm"
file_download_link = "https://docs.google.com/uc?export=download&id=" + file_id 

!wget -q -O rnaseq_raw_counts.txt --no-check-certificate "$file_download_link"

In [ ]:
!ls ## run this command ('ls') to list the files in the current directory

### Read in the data with pandas

Now we can start with python!  Enter code below to read the file into python as a pandas data matrix object. Call this variable `rna_data`.  To read in the data correctly, note that:
+ the separator between fields is a tab
+ the first row is column headers, and the first column is row headers

In [ ]:
#type your code here


### Describe the data

What are the dimensions of the data matrix we just imported?

In [ ]:
#type your code here


That's pretty big. We want to be careful not to try to look at the whole matrix at once because it will fill up the screen with endless numbers. Write code below to print just the first 10 rows and 10 columns to see what a little piece of the data looks like.



In [ ]:
#type your code here


In this matrix the genes are listed in rows and samples are in the columns.  

The gene IDs (row headers) all have the form 'Cz' followed by a number, then the letter 'g', then another number.  'Cz' indicates that the gene is from the species _Chromochloris zofingiensis_, the first number indicates which chromosome the gene is on, and the second number is a randomly assigned ID number.  

The sample names (column headers) tell us whether each algae was grown in medium light (ML) or high light (HL), and how many hours the algae was in the light before they collected a sample (eg. 0.5h, 1h, 12h).  Multiple algae replicates were sampled at each timepoint - the last number in each sample name is an ID number for the replicate.

How many genes are there in the full data matrix? How many samples?

_add your answer here_

Each of the numbers in the data matrix represents the number of reads that mapped to a specific gene in a specific sample.  What data type should the matrix be (strings? integers? floats?)? 

_add your answer here_

### Normalizing the data 

#### By Read Depth 

Take a look at the numbers in just the 10x10 subset of the matrix you printed above.  Which sample has the *fewest* reads mapping to gene `Cz01g00060`? What about the gene `Cz01g00070`?

_add your answers here_

It's possible that the same sample has the fewest reads for both genes because that sample actually expresses less of those genes than the other samples... but it also could be because we just sequenced fewer reads from that sample overall.  **The total number of reads sequenced from a sample (from all genes) is known as a sample's read depth**.  We need to account for this before making any comparisons of gene expression between samples.

Think about how you would calculate the read depth for each sample using the data in the matrix. Write code in the box below and store the values in a variable called `read_depth`.

In [ ]:
#type your code here
## hint: one way to do this uses a for-loop - but there are other ways too!


Now make a plot so we can see the range of read depths across all samples.

In [ ]:
#type your code here
### hint: this could take many forms - try a scatterplot first


There are some big differences between samples. The deepest-sequenced sample has ~8 times the number of reads as the most shallowly-sequenced sample. So we need to **normalize** the data matrix by read depth.  The new matrix will not have the *number* of reads that aligned to each gene, but rather the *fraction* of reads from each sample that aligned to each gene.

Below is a function to normalize either the rows or columns of a data matrix by a vector (in our case, this vector is the read depth of each sample).  It takes three pieces of data (ie. parameters) which are listed in the comment lines below.  

Pass the `rna_data` matrix and your `read_depth` variable to the function as parameters, along with whether you are trying to normalize the rows or columns (which ones represent samples?) to get a normalized matrix. Name the normalized matrix `rna_data_norm`

In [ ]:
### this function has been written for you - no need to do anything here besides run the code/
### use the next box to pass in your parameters and run the function

def normalizeData(data_matrix,norm_values,norm_dimension):

  ## data_matrix is a numeric matrix of data to be normalized
  ## norm_values is a 1-dimension vector of normalization factors
  ## norm_dimension must be either "rows" or "columns"

  import numpy as np
  if norm_dimension == "rows":
    if len(norm_values) != len(data_matrix.index):
      print("norm_values is the wrong size!")
      return data_matrix 
    return np.transpose(np.transpose(data_matrix)/norm_values)
  if norm_dimension == "columns": 
    if len(norm_values) != len(data_matrix.columns):
      print("norm_values is the wrong size!")
      return data_matrix
    return data_matrix / norm_values


In [ ]:
#type your code here


#### By Gene Length

Great! Now let's look at the normalized values for all samples for just the first 100 genes as a **heatmap**. A heatmap displays a matrix of numeric values as a grid where each number maps to a different color shade. An easy way to view a matrix as a heatmap is with the heatmap function from the seaborn package.  

In [ ]:
### just run this code - don't worry about writing anything new (unless you want to try a different plotting tool!)
import seaborn as sns
ax = sns.heatmap(rna_data_norm.iloc[0:100,:], linewidth=0)


We can see that a couple rows (genes) have high values (darker colors) for many samples, and many genes have basically zero or very low values for all of the samples.  

Again, this could be because those few dark-colored rows represent genes that are highly expressed in all samples - *but* it could also be because those genes are *longer* than the other genes, and we are sequencing many reads from the same long mRNA molecule.

So, if we really want to have comparable values across genes and across samples, we also need to normalize our matrix by gene lengths.   

First, let's download information about the genes, and take a look at the first few rows of the gene data table.

In [ ]:
### run this code as-is
file_id = "16LXBxn0hOqBgARJLyk8aHev3OjJbqflG"
file_download_link = "https://docs.google.com/uc?export=download&id=" + file_id 

!wget -q -O gene_info.txt --no-check-certificate "$file_download_link"

In [ ]:
#type your code here to read in the gene info table - call the variable gene_info
## separator is a tab, there are row and column headers


In [ ]:
### type code here to look at the first few lines of gene_info


First things first - are the genes in the `gene_info` table all in the same order as the genes in the `rna_data_norm` matrix?  (**when you are combining data from two tables, it's always important to check that they line up - skipping this step can create big problems down the road!**)

In [ ]:
## write code to check this here


In [ ]:
### now write code to normalize the rows of the data matrix by the 'length' column of gene_info
### call the new data matrix 'rna_data_norm2'


### Visualizing the data

Now that we have normalized data, we can see how it looks all together!

Principal Component Analysis (PCA) is an important technique in many areas of research (from biology to sociology!) that lets you see whether your data contains groups of samples that all have similar profiles (in our case - similar gene expression profiles).

The way PCA does this is somewhat complicated, but the main result is that we will be able to make a 2-D plot where each sample is a dot, and samples that have similar gene expression profiles will be close to each other on the plot.

Run the following code block as-is to make the PCA plot. (No need to code anything yourself for now - but someday you will be able to code this part on your own!)


In [ ]:
### load packages
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np

### this code filters out any rows that are all 'zero', then applies a log2 transformation and standardizes the data 
### (basically this helps squash extreme values so different genes are treated equally)
filter=rna_data_norm2.sum( axis=1)>0
rna_data_filtered=rna_data_norm2[filter]
really_small_number = rna_data_filtered[rna_data_filtered > 0].min().min()
rna_data_log2=(rna_data_filtered + really_small_number).applymap(np.log2)
rna_data_log2 = StandardScaler().fit_transform(rna_data_log2)

### run PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(rna_data_log2.transpose())
rna_data_pca = pd.DataFrame(data = principalComponents
             , columns = ['pc1', 'pc2'])

### parse sample names to get find out which timepoint and light-treatment each was from
rna_data_pca['samplename']=rna_data_filtered.columns
rna_data_pca[['samplename','replicate']]=rna_data_pca.samplename.str.split("h",expand=True)
rna_data_pca[['light','timepoint']]=rna_data_pca.samplename.str.split(".",expand=True,  n=1)
rna_data_pca.timepoint = pd.Categorical(rna_data_pca.timepoint)
rna_data_pca.light = pd.Categorical(rna_data_pca.light)

### now plot the first two principal components
fig = plt.figure(figsize = (5,5))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('PC 1', fontsize = 15)
ax.set_ylabel('PC 2', fontsize = 15)
ax.set_title('Principal Component Plot', fontsize = 20)

light_types=['ML','HL']
light_shapes=['^','o']

timepoint_types=rna_data_pca.timepoint.cat.categories[[0,1,2,4,5,3]]
timepoint_colors=['red','orange','yellow','green','blue','purple']

for ltype, lshape in zip(light_types,light_shapes):
    for ttype, tcolor in zip(timepoint_types,timepoint_colors):
      indicesToKeep = (rna_data_pca['light'] == ltype) & (rna_data_pca['timepoint'] == ttype)
      df=rna_data_pca.loc[indicesToKeep]
      ax.scatter(df.pc1, -1*df.pc2, s=50,c=tcolor, marker=lshape, label=ltype+"_"+ttype+"h",edgecolors='black')
ax.legend(bbox_to_anchor=(1.05, 1), loc=2,ncol=2, borderaxespad=0.)

### Drawing conclusions from the results

Judging from the principal component plot you just made, which would you say matters more for gene expression - sample treatment (ie. whether they were exposed to medium or high light), or the timepoint of sample collection?  

Put another way - are samples generally more similar to other samples from the same treatment, or other samples from the same timepoint?

_add your answer here_

At which timepoint would you say that the expression profiles of samples from the two treatments are *most different*?

_add your answer here_

Now take a look at figure 5A in the paper describing the experiment.  How did we do?

![alt text](https://www.pnas.org/content/pnas/114/21/E4296/F5.large.jpg)